In [ ]:
import os
import time
import math
import magenta.music as mm
import tensorflow as tf
from magenta.models.polyphony_rnn import polyphony_sequence_generator
from magenta.models.shared import sequence_generator_bundle
from magenta.music import DEFAULT_QUARTERS_PER_MINUTE
from magenta.music.protobuf.generator_pb2 import GeneratorOptions
from magenta.music.protobuf.music_pb2 import NoteSequence
from visual_midi import Plotter

print(tf.test.gpu_device_name())

In [ ]:
def generate(bundle_name, seq_generator, gen_id, primer, qpm = DEFAULT_QUARTERS_PER_MINUTE, 
             condition_on_primer = False, inject_primer_during_generation = False, total_steps = 64, 
             temp = 1.0, beam_size = 1, branch_factor = 1, steps_per_iteration = 1):
    
    mm.notebook_utils.download_bundle(bundle_name, "bundles")
    bundle = sequence_generator_bundle.read_bundle_file(os.path.join("bundles", bundle_name))
    
    generator_map = seq_generator.get_generator_map()
    generator = generator_map[gen_id](checkpoint = None, bundle = bundle)
    generator.initialize()
    
    if primer:
        primer_seq = mm.midi_io.midi_file_to_note_sequence(primer)
    else:
        primer_seq = NoteSequence()
        
    if primer_seq.tempos:
        if len(primer_seq.tempos) > 1:
            raise Exception('Monophonic Melody Generation. No support for multiple tempos')
        qpm = primer_seq.tempos[0].qpm
        
    secs_per_step = 60.0 / (qpm * getattr(generator, 'steps_per_quarter', 4))
    
    primer_seq_steps = math.ceil(primer_seq.total_time / secs_per_step)
    primer_seq_time = primer_seq_steps * secs_per_step
    primer_end_ad = (0.00001 if primer_seq_time > 0 else 0)
    primer_start_time = 0
    primer_end_time = primer_start_time + primer_seq_time - primer_end_ad
    
    gen_steps = total_steps - primer_seq_steps
    
    if gen_steps <= 0:
        raise Exception('Total steps too small : {}. Need at least 1 bar bigger than primer : {}'.format(
        total_steps, primer_seq_steps))
    
    gen_time = gen_steps * secs_per_step
    gen_start_time = primer_end_time
    gen_end_time = gen_start_time + gen_time + primer_end_ad
    
    print('Primer start : {} \t Primer end : {}'.format(primer_start_time, primer_end_time))
    print('Generation start : {} \t Generation end : {}'.format(gen_start_time, gen_end_time))
    
    generator_options = GeneratorOptions()
    generator_options.args['temperature'].float_value = temp
    generator_options.args['beach_size'].int_value = beam_size
    generator_options.args['branch_factor'].int_value = branch_factor
    generator_options.args['steps_per_iteration'].int_value = steps_per_iteration
    generator_options.args['condition_on_primer'].bool_value = condition_on_primer
    generator_options.args['no_inject_primer_during_generation'].bool_value = not inject_primer_during_generation
    generator_options.generate_sections.add(start_time = gen_start_time, end_time = gen_end_time)
    
    seq = generator.generate(primer_seq, generator_options)
    
    # Write midi file
    date_time = time.strftime('%Y-%m-%d_%H%M%S')
    gen_name = str(generator.__class__).split('.')[2]
    midi = '%s_%s_%s.mid' % (gen_name, gen_id, date_time)
    midi_path = os.path.join('output', midi)
    mm.midi_io.note_sequence_to_midi_file(seq, midi_path)
    print('Generated midi : {}'.format(os.path.abspath(midi_path)))
    
    # Generate plot
    date_time = time.strftime('%Y-%m-%d_%H%M%S')
    gen_name = str(generator.__class__).split('.')[2]
    plot = '%s_%s_%s.html' % (gen_name, gen_id, date_time)
    plot_path = os.path.join('output', plot)
    
    pretty_midi = mm.midi_io.note_sequence_to_pretty_midi(seq)
    plotter = Plotter()
    plotter.show(pretty_midi, plot_path)
    print('Generated plot : {}'.format(os.path.abspath(plot_path)))
    
    return seq

In [ ]:
generate("polyphony_rnn.mag", polyphony_sequence_generator, "polyphony",
         condition_on_primer = False, inject_primer_during_generation = False, temp = 0.9,
         primer = "primers/Fur_Elisa_Beethoveen_Polyphonic.mid")